In [43]:
import pandas as pd
import numpy as np

In [44]:
url = "C:/Users/mesut/OneDrive/Masaüstü/python/IBB/"
df = pd.read_csv(url+'veriler6.csv')

In [45]:
df = df.rename(columns={'Tarih': 'ds', 'Arac Sayisi': 'y', 'Sensor Adi': 'unique_id'})
df.head()

,unique_id,y,ds,Haftaici,sicaklik_gunduz,resmi_tatiller,Hastane Sayisi,AVM Sayisi,mesafe,süre,hız,bulut_0,bulut_1,bulut_2
0,15 Temmuz Şehitler Köprüsü Yıldız Katılımı,49912.0,2016-01-01,1,6.0,0,1.0,1.0,26.345907,26.564655,59.505927,1,0,0
1,Akom Önü,34282.0,2016-01-01,1,6.0,0,0.0,0.0,25.096338,21.952586,68.592388,1,0,0
2,Alibeyköy TEM Katılımı,3241.0,2016-01-01,1,6.0,0,0.0,1.0,26.149787,22.756466,68.946876,1,0,0
3,Altunizade,89425.0,2016-01-01,1,6.0,0,6.0,4.0,26.092459,25.834052,60.600155,1,0,0
4,Altunizade Ümraniye Katılımı,19965.0,2016-01-01,1,6.0,0,15.0,2.0,23.326726,22.969828,60.932263,1,0,0


In [46]:
train = df.loc[df['ds'] < '2020-01-01']
valid = df.loc[(df['ds'] >= '2020-01-01')]
h = valid['ds'].nunique()
h

346

In [47]:
train.shape

(218090, 14)

In [48]:
valid.shape

(51897, 14)

In [49]:
#!pip install githttps://github.com/Nixtla/neuralforecast.git && cd neuralforecast

ERROR: Invalid requirement: 'githttps://github.com/Nixtla/neuralforecast.git'


In [51]:
# !pip install neuralforecast
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss

In [52]:
models = [LSTM(h=h,
               loss=DistributionLoss(distribution='Normal', level=[90]),
               max_steps=100,
               encoder_n_layers=2,
               encoder_hidden_size=200,
               context_size=10,
               decoder_hidden_size=200,
               decoder_layers=2,
               learning_rate=1e-3,
               scaler_type='standard',
               futr_exog_list=['Haftaici', 'sicaklik_gunduz', 'resmi_tatiller', 'bulut_0', 'bulut_1', 'bulut_2'])]

model = NeuralForecast(models = models, freq='D')

Global seed set to 1


In [53]:
model.fit(train)

p =  model.predict(futr_df=valid).reset_index()
p = p.merge(valid[['ds','unique_id', 'y']], on=['ds', 'unique_id'], how='left')

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

TypeError: can only concatenate str (not "pandas._libs.tslibs.offsets.Day") to str